#### Qual embedding?

In [1]:
TABLE = "rag_embeddings_s"

https://python.langchain.com/docs/tutorials/rag/

BigQuery table llm-studies.blog_embeddings.rag_embeddings initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=llm-studies&ws=!1m5!1m4!4m3!1sllm-studies!2sblog_embeddings!3srag_embeddings


In [12]:
graph_builder = StateGraph(MessagesState)
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [13]:
    response = graph.invoke(
        {"messages": [{"role": "user", "content": "Dennett é um eliminativista?"}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]},
        stream_mode="values",
    )

In [14]:
print(response.get('messages')[4].content)

Sim, Dennett é frequentemente associado ao eliminativismo, a visão de que conceitos como "qualia" e "experiência subjetiva" podem e devem ser eliminados da nossa linguagem e teoria da mente.


In [36]:
print(len(response.get('messages')))

5


In [32]:
for amessage in response.get('messages'):
    print(amessage)
    print("/n/n/n")
    


content='Dennett é um eliminativista?' additional_kwargs={} response_metadata={} id='992baffe-f620-43fb-9f98-c3056f0a5f4e'
/n/n/n
content='Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos' additional_kwargs={} response_metadata={} id='be2b04da-aacb-4517-bfff-29243ae394f3'
/n/n/n
content='' additional_kwargs={'function_call': {'name': 'retrieve', 'arguments': '{"query": "posi\\u00e7\\u00e3o de Daniel Dennett sobre o eliminativismo"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 44, 'candidates_token_count': 12, 'total_token_count': 56, 'prompt_tokens_details': [{'modality': 1, 'token_count': 44}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 12}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.07389294604460399, 'model_name': 'gemini-2.0-flash-001'} id='run-0f88446e-5c9c-4663-8

In [19]:
input_message = "Dennett é um eliminativista?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ System Message ================================

Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos
================================== Ai Message ==================================
Tool Calls:
  retrieve (6b6006c4-e217-4d33-9752-523192dff898)
 Call ID: 6b6006c4-e217-4d33-9752-523192dff898
  Args:
    query: posição de Daniel Dennett sobre o eliminativismo
================================= Tool Message =================================
Name: retrieve

Source: {'doc_id': '701792560a3d49af8ed3e872610183de', 'source': 'https://www.reflexoesdofilosofo.blog.br/2025/01/heterofenomenologia.html', 'score': 0.6008632452310769}
Content: constitui a essência da consciência. Ele propõe uma visão alternativa, que é baseada em uma abordagem materialista e funcionalista, onde a experiência subjetiva é vista como um conjunto de funções cerebrais e comportamentais, em vez de uma entidade metafísi

In [18]:
input_message = "Me fale quem é eliminativista"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ System Message ================================

Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos
================================== Ai Message ==================================
Tool Calls:
  retrieve (1f070483-f4e6-42b1-99c8-432e9a72735c)
 Call ID: 1f070483-f4e6-42b1-99c8-432e9a72735c
  Args:
    query: eliminativismo
================================= Tool Message =================================
Name: retrieve

Source: {'doc_id': 'fb7fcafa9ffe47269dbad63df4a0eb18', 'source': 'https://www.reflexoesdofilosofo.blog.br/2023/02/descritivismo.html', 'score': 0.6619441515696384}
Content: Descritivismo

Source: {'doc_id': '9afdd777f93a40b5a366da0d475c7ebb', 'source': 'https://www.reflexoesdofilosofo.blog.br/2022/12/referencialismo.html', 'score': 0.7284535514220247}
Content: Referencialismo
================================== Ai Message ==================================

Não tenho

In [12]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")